In [1]:
from marubatsu import Marubatsu
from ai import ai2, ai2s

mb = Marubatsu()
%timeit ai2(mb)
%timeit ai2s(mb)

2.3 μs ± 24.8 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
23.1 μs ± 878 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [2]:
from marubatsu import ListBoard

def __init__(self, boardclass=ListBoard, board_size=3, *args, **kwargs):
    # ゲーム盤のデータ構造を定義するクラス
    self.boardclass = boardclass
    # ゲーム盤の縦横のサイズ
    self.BOARD_SIZE = board_size
    # boardclass のパラメータ
    self.args = args
    self.kwargs = kwargs
    # 〇×ゲーム盤を再起動するメソッドを呼び出す
    self.restart()
    
Marubatsu.__init__ = __init__

In [3]:
def place_mark(self, x, y, mark):
    if 0 <= x < self.BOARD_SIZE and 0 <= y < self.BOARD_SIZE:
        if self.board.getmark(x, y) == Marubatsu.EMPTY:
            self.board.setmark(x, y, mark)
            return True
        else:
            print("(", x, ",", y, ") のマスにはマークが配置済です")
            return False
    else:
        print("(", x, ",", y, ") はゲーム盤の範囲外の座標です")
        return False   
    
Marubatsu.place_mark = place_mark

In [4]:
def move(self, x, y, check_coord=False):
    if not check_coord:
        self.board.setmark(x, y, self.turn)
    elif not self.place_mark(x, y, self.turn):
        return 

    self.last_turn = self.turn
    self.turn = Marubatsu.CROSS if self.turn == Marubatsu.CIRCLE else Marubatsu.CIRCLE  
    self.move_count += 1
    self.last_move = x, y
    self.status = self.board.judge(self.last_turn, self.last_move, self.move_count)
    if len(self.records) <= self.move_count:            
        self.records.append(self.last_move)
    else:
        self.records[self.move_count] = self.last_move
        self.records = self.records[0:self.move_count + 1]
        
Marubatsu.move = move

In [5]:
def unmove(self):
    if self.move_count > 0:
        x, y = self.last_move
        if self.move_count == 0:
            self.last_move = (-1, -1)
        self.move_count -= 1
        self.turn, self.last_turn = self.last_turn, self.turn
        self.status = Marubatsu.PLAYING
        x, y = self.records.pop()
        self.board.setmark(x, y, Marubatsu.EMPTY)       
        self.last_move = self.records[-1]    
        
Marubatsu.unmove = unmove

In [6]:
def play_loop(self, mb_gui, params=None):
    if params is None:
        params = [{}, {}]
    
    ai = self.ai
    verbose = self.verbose
    gui = self.gui
    
    # ゲームの決着がついていない間繰り返す
    while self.status == Marubatsu.PLAYING:
        # 現在の手番を表す ai のインデックスを計算する
        index = 0 if self.turn == Marubatsu.CIRCLE else 1
        # ゲーム盤の表示
        if verbose:
            if gui:
                # AI どうしの対戦の場合は画面を描画しない
                if ai[0] is None or ai[1] is None:
                    mb_gui.update_gui()
                # 手番を人間が担当する場合は、play メソッドを終了する
                if ai[index] is None:
                    return
            else:
                print(self)
                
        # ai が着手を行うかどうかを判定する
        if ai[index] is not None:
            x, y = ai[index](self, **params[index])
            self.move(x, y)
        else:
            # キーボードからの座標の入力
            coord = input("x,y の形式で座標を入力して下さい。exit を入力すると終了します")
            # "exit" が入力されていればメッセージを表示して関数を終了する
            if coord == "exit":
                print("ゲームを終了します")
                return       
            # x 座標と y 座標を要素として持つ list を計算する
            xylist = coord.split(",")
            # xylist の要素の数が 2 ではない場合
            if len(xylist) != 2:
                # エラーメッセージを表示する
                print("x, y の形式ではありません")
                # 残りの while 文のブロックを実行せずに、次の繰り返し処理を行う
                continue
            x, y = xylist
            try:
                self.move(int(x), int(y), check_coord=True)
            except:
                print("整数の座標を入力して下さい")

    # 決着がついたので、ゲーム盤を表示する
    if verbose:
        if gui:
            mb_gui.update_gui()
        else:
            print(self)
            
    return self.status

Marubatsu.play_loop = play_loop

In [7]:
mb = Marubatsu()
mb.play(ai=[None, None])

Turn o
...
...
...

Turn x
O..
...
...

( 0 , 0 ) のマスにはマークが配置済です
Turn x
O..
...
...

( 3 , 5 ) はゲーム盤の範囲外の座標です
Turn x
O..
...
...

整数の座標を入力して下さい
Turn x
O..
...
...

ゲームを終了します


In [8]:
mb = Marubatsu()
mb.move(0, 0)
print(mb)
mb.move(0, 0)
print(mb)

Turn x
O..
...
...

Turn o
X..
...
...



In [9]:
mb.move(3, 5)

IndexError: list index out of range

In [10]:
mb = Marubatsu()
mb.move(0, 0, check_coord=True)
print(mb)
mb.move(0, 0, check_coord=True)
print(mb)
mb.move(3, 5, check_coord=True)

Turn x
O..
...
...

( 0 , 0 ) のマスにはマークが配置済です
Turn x
O..
...
...

( 3 , 5 ) はゲーム盤の範囲外の座標です


In [11]:
def cmove(self, x, y):
    self.move(x, y, check_coord=True)
    
Marubatsu.cmove = cmove

In [12]:
mb = Marubatsu()
mb.cmove(0, 0)
print(mb)
mb.cmove(0, 0)
print(mb)
mb.cmove(3, 5)

Turn x
O..
...
...

( 0 , 0 ) のマスにはマークが配置済です
Turn x
O..
...
...

( 3 , 5 ) はゲーム盤の範囲外の座標です


In [13]:
mb.cmove("a", "b")

TypeError: '<=' not supported between instances of 'int' and 'str'

In [14]:
def place_mark(self, x, y, mark):
    try:
        x = int(x)
        y = int(y)
    except:
        print("整数の座標を入力して下さい")
        return False

    if 0 <= x < self.BOARD_SIZE and 0 <= y < self.BOARD_SIZE:
        if self.board.getmark(x, y) == Marubatsu.EMPTY:
            self.board.setmark(x, y, mark)
            return True
        else:
            print("(", x, ",", y, ") のマスにはマークが配置済です")
            return False
    else:
        print("(", x, ",", y, ") はゲーム盤の範囲外の座標です")
        return False   
    
Marubatsu.place_mark = place_mark

In [15]:
mb.cmove("a", "b")

整数の座標を入力して下さい


In [16]:
def play_loop(self, mb_gui, params=None):
    if params is None:
        params = [{}, {}]
    
    ai = self.ai
    verbose = self.verbose
    gui = self.gui
    
    # ゲームの決着がついていない間繰り返す
    while self.status == Marubatsu.PLAYING:
        # 現在の手番を表す ai のインデックスを計算する
        index = 0 if self.turn == Marubatsu.CIRCLE else 1
        # ゲーム盤の表示
        if verbose:
            if gui:
                # AI どうしの対戦の場合は画面を描画しない
                if ai[0] is None or ai[1] is None:
                    mb_gui.update_gui()
                # 手番を人間が担当する場合は、play メソッドを終了する
                if ai[index] is None:
                    return
            else:
                print(self)
                
        # ai が着手を行うかどうかを判定する
        if ai[index] is not None:
            x, y = ai[index](self, **params[index])
            self.move(x, y)
        else:
            # キーボードからの座標の入力
            coord = input("x,y の形式で座標を入力して下さい。exit を入力すると終了します")
            # "exit" が入力されていればメッセージを表示して関数を終了する
            if coord == "exit":
                print("ゲームを終了します")
                return       
            # x 座標と y 座標を要素として持つ list を計算する
            xylist = coord.split(",")
            # xylist の要素の数が 2 ではない場合
            if len(xylist) != 2:
                # エラーメッセージを表示する
                print("x, y の形式ではありません")
                # 残りの while 文のブロックを実行せずに、次の繰り返し処理を行う
                continue
            x, y = xylist
            self.cmove(x, y)

    # 決着がついたので、ゲーム盤を表示する
    if verbose:
        if gui:
            mb_gui.update_gui()
        else:
            print(self)
            
    return self.status

Marubatsu.play_loop = play_loop

In [17]:
mb.play(ai=[None, None])


Turn o
...
...
...

整数の座標を入力して下さい
Turn o
...
...
...

ゲームを終了します


In [ ]:
from util import gui_play

gui_play()

In [19]:
from marubatsu import Marubatsu_GUI
from tkinter import Tk, filedialog
import pickle
from datetime import datetime
import math

def create_event_handler(self):
    # 乱数の種のチェックボックスのイベントハンドラを定義する
    def on_checkbox_changed(changed):
        self.update_widgets_status()
        
    self.checkbox.observe(on_checkbox_changed, names="value")

    # 開く、保存ボタンのイベントハンドラを定義する
    def on_load_button_clicked(b=None):
        path = filedialog.askopenfilename(filetypes=[("〇×ゲーム", "*.mbsav")],
                                        initialdir="save")
        if path != "":
            with open(path, "rb") as f:
                data = pickle.load(f)
                self.mb.records = data["records"]
                self.mb.ai = data["ai"]
                self.params = data["params"] if "params" in data else [ {}, {} ]
                if "names" in data:
                    names = data["names"]
                else:
                    names = [ "人間" if self.mb.ai[i] is None else self.mb.ai[i].__name__ for i in range(2)]                       
                options = self.dropdown_list[0].options.copy()
                for i in range(2):
                    value = (self.mb.ai[i], self.params[i]) 
                    if not value in options.values():
                        options[names[i]] = value
                for i in range(2):
                    self.dropdown_list[i].options = options
                    self.dropdown_list[i].value = (self.mb.ai[i], self.params[i])            
                status_options = options.copy()
                status_options["手番の AI"] = ("Auto", None)
                self.status_dropdown.options = status_options
                change_step(data["move_count"])
                if data["seed"] is not None:                   
                    self.checkbox.value = True
                    self.inttext.value = data["seed"]
                else:
                    self.checkbox.value = False
                    
    def on_save_button_clicked(b=None):
        names = [ self.dropdown_list[i].label for i in range(2) ]     
        timestr = datetime.now().strftime("%Y年%m月%d日 %H時%M分%S秒")
        fname = f"{names[0]} VS {names[1]} {timestr}"
        path = filedialog.asksaveasfilename(filetypes=[("〇×ゲーム", "*.mbsav")],
                                            initialdir="save", initialfile=fname,
                                            defaultextension="mbsav")
        if path != "":
            with open(path, "wb") as f:
                data = {
                    "records": self.mb.records,
                    "move_count": self.mb.move_count,
                    "ai": self.mb.ai,
                    "params": self.params,
                    "names": names,
                    "seed": self.inttext.value if self.checkbox.value else None
                }
                pickle.dump(data, f)
                
    def on_show_tree_button_clicked(b=None):
        self.show_subtree = not self.show_subtree
        self.mbtree_gui.vbox.layout.display = None if self.show_subtree else "none"
        self.update_gui()
        
    def on_reset_tree_button_clicked(b=None):
        self.update_gui()
                
    def on_help_button_clicked(b=None):
        self.help.layout.display = "none" if self.help.layout.display is None else None

    self.load_button.on_click(on_load_button_clicked)
    self.save_button.on_click(on_save_button_clicked)
    self.show_tree_button.on_click(on_show_tree_button_clicked)
    self.reset_tree_button.on_click(on_reset_tree_button_clicked)
    self.help_button.on_click(on_help_button_clicked)
    
    def on_show_status_button_clicked(b=None):
        self.show_status = not self.show_status
        self.update_gui()

    def on_status_dropdown_changed(changed):
        self.update_gui()

    def on_size_slider_changed(changed):
        self.size = changed["new"]
        self.fig.set_figwidth(self.size)
        self.fig.set_figheight(self.size)
        self.update_gui()

    self.show_status_button.on_click(on_show_status_button_clicked)
    self.status_dropdown.observe(on_status_dropdown_changed, names="value")
    self.size_slider.observe(on_size_slider_changed, names="value")

    # 変更ボタンのイベントハンドラを定義する
    def on_change_button_clicked(b):
        for i in range(2):
            self.mb.ai[i], self.params[i] = self.dropdown_list[i].value
        self.mb.play_loop(self, self.params)

    # リセットボタンのイベントハンドラを定義する
    def on_reset_button_clicked(b=None):
        # 乱数の種のチェックボックスが ON の場合に、乱数の種の処理を行う
        if self.checkbox.value:
            random.seed(self.inttext.value)
        self.mb.restart()
        self.output.clear_output()
        on_change_button_clicked(b)

    # 待ったボタンのイベントハンドラを定義する
    def on_undo_button_clicked(b=None):
        if self.mb.move_count >= 2 and self.mb.move_count == len(self.mb.records) - 1:
            self.mb.move_count -= 2
            self.mb.records = self.mb.records[0:self.mb.move_count+1]
            self.mb.change_step(self.mb.move_count)
            self.update_gui()
        
    # イベントハンドラをボタンに結びつける
    self.change_button.on_click(on_change_button_clicked)
    self.reset_button.on_click(on_reset_button_clicked)   
    self.undo_button.on_click(on_undo_button_clicked)   

    # step 手目の局面に移動する
    def change_step(step):
        self.mb.change_step(step)
        # 描画を更新する
        self.update_gui()        

    def on_first_button_clicked(b=None):
        change_step(0)

    def on_prev_button_clicked(b=None):
        change_step(self.mb.move_count - 1)

    def on_next_button_clicked(b=None):
        change_step(self.mb.move_count + 1)
        
    def on_last_button_clicked(b=None):
        change_step(len(self.mb.records) - 1)

    def on_slider_changed(changed):
        if self.mb.move_count != changed["new"]:
            change_step(changed["new"])
        
    self.first_button.on_click(on_first_button_clicked)
    self.prev_button.on_click(on_prev_button_clicked)
    self.next_button.on_click(on_next_button_clicked)
    self.last_button.on_click(on_last_button_clicked)
    self.slider.observe(on_slider_changed, names="value")

    # ゲーム盤の上でマウスを押した場合のイベントハンドラ
    def on_mouse_down(event):
        # Axes の上でマウスを押していた場合のみ処理を行う
        if event.inaxes and self.mb.status == Marubatsu.PLAYING:
            x = math.floor(event.xdata)
            y = math.floor(event.ydata)
            with self.output:
                self.mb.cmove(x, y)             
            # 次の手番の処理を行うメソッドを呼び出す
                self.mb.play_loop(self, self.params)

    # ゲーム盤を選択した状態でキーを押した場合のイベントハンドラ
    def on_key_press(event):
        keymap = {
            "up": on_first_button_clicked,
            "left": on_prev_button_clicked,
            "right": on_next_button_clicked,
            "down": on_last_button_clicked,
            "0": on_undo_button_clicked,
            "enter": on_reset_button_clicked,            
            "-": on_load_button_clicked,            
            "l": on_load_button_clicked,            
            "+": on_save_button_clicked,            
            "s": on_save_button_clicked,            
            "*": on_help_button_clicked,            
            "h": on_help_button_clicked,            
        }
        if event.key in keymap:
            keymap[event.key]()
        else:
            try:
                num = int(event.key) - 1
                event.inaxes = True
                event.xdata = num % 3
                event.ydata = 2 - (num // 3)
                on_mouse_down(event)
            except:
                pass
            
    # fig の画像イベントハンドラを結び付ける
    self.fig.canvas.mpl_connect("button_press_event", on_mouse_down)     
    self.fig.canvas.mpl_connect("key_press_event", on_key_press)    

Marubatsu_GUI.create_event_handler = create_event_handler

In [20]:
gui_play()

In [21]:
from ai import ai_match, ai14s, ai_abs_dls
import random

def benchmark(mbparams={}, match_num=50000, seed=0):
    if seed is not None:
        random.seed(seed)       
        
    ai_match(ai=[ai2, ai2], match_num=match_num, mbparams=mbparams)   
    ai_match(ai=[ai14s, ai2], match_num=match_num, mbparams=mbparams)

    mb = Marubatsu(**mbparams)
    eval_params = {"minimax": True}
    print("ai_abs_dls")
    %timeit ai_abs_dls(mb, eval_func=ai14s, eval_params=eval_params, use_tt=True, maxdepth=8)

In [22]:
from marubatsu import List1dBoard

for boardclass in [ListBoard, List1dBoard]:
    for count_linemark in [False, True]:
        print(f"boardclass: {boardclass.__name__}, count_linemark {count_linemark}")
        benchmark(mbparams={"boardclass": boardclass, "count_linemark": count_linemark})
        print()

boardclass: ListBoard, count_linemark False
ai2 VS ai2


100%|██████████| 50000/50000 [00:04<00:00, 12391.16it/s]


count     win    lose    draw
o       29454   14352    6194
x       14208   29592    6200
total   43662   43944   12394

ratio     win    lose    draw
o       58.9%   28.7%   12.4%
x       28.4%   59.2%   12.4%
total   43.7%   43.9%   12.4%

ai14s VS ai2


100%|██████████| 50000/50000 [00:51<00:00, 969.45it/s] 


count     win    lose    draw
o       49446       0     554
x       44043       0    5957
total   93489       0    6511

ratio     win    lose    draw
o       98.9%    0.0%    1.1%
x       88.1%    0.0%   11.9%
total   93.5%    0.0%    6.5%

ai_abs_dls
17.4 ms ± 143 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)

boardclass: ListBoard, count_linemark True
ai2 VS ai2


...

count     win    lose    draw
o       49446       0     554
x       44043       0    5957
total   93489       0    6511

ratio     win    lose    draw
o       98.9%    0.0%    1.1%
x       88.1%    0.0%   11.9%
total   93.5%    0.0%    6.5%

ai_abs_dls
17.6 ms ± 477 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)

boardclass: List1dBoard, count_linemark True
ai2 VS ai2


100%|██████████| 50000/50000 [00:04<00:00, 12300.84it/s]


count     win    lose    draw
o       29454   14352    6194
x       14208   29592    6200
total   43662   43944   12394

ratio     win    lose    draw
o       58.9%   28.7%   12.4%
x       28.4%   59.2%   12.4%
total   43.7%   43.9%   12.4%

ai14s VS ai2


100%|██████████| 50000/50000 [00:26<00:00, 1875.35it/s]


count     win    lose    draw
o       49446       0     554
x       44043       0    5957
total   93489       0    6511

ratio     win    lose    draw
o       98.9%    0.0%    1.1%
x       88.1%    0.0%   11.9%
total   93.5%    0.0%    6.5%

ai_abs_dls
18.6 ms ± 1.48 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)



In [23]:
benchmark(mbparams={"boardclass": ListBoard, "count_linemark": False}, seed=1)

ai2 VS ai2


100%|██████████| 50000/50000 [00:04<00:00, 11799.85it/s]


count     win    lose    draw
o       29318   14501    6181
x       14278   29306    6416
total   43596   43807   12597

ratio     win    lose    draw
o       58.6%   29.0%   12.4%
x       28.6%   58.6%   12.8%
total   43.6%   43.8%   12.6%

ai14s VS ai2


100%|██████████| 50000/50000 [00:51<00:00, 971.35it/s] 


count     win    lose    draw
o       49524       0     476
x       44126       0    5874
total   93650       0    6350

ratio     win    lose    draw
o       99.0%    0.0%    1.0%
x       88.3%    0.0%   11.7%
total   93.7%    0.0%    6.3%

ai_abs_dls
17.8 ms ± 341 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [24]:
%timeit 1 + 2

10.1 ns ± 0.224 ns per loop (mean ± std. dev. of 7 runs, 100,000,000 loops each)


In [25]:
import timeit 

number = 100000000
repeat = 7
result = timeit.repeat(stmt="1 + 2", number=number, repeat=repeat)
print(result)

[1.14166110008955, 1.1156697981059551, 1.1045803017914295, 1.1856296993792057, 1.1052100993692875, 1.1550979986786842, 1.1028691977262497]


In [26]:
result = [time / number for time in result]
print(result)

[1.14166110008955e-08, 1.115669798105955e-08, 1.1045803017914295e-08, 1.1856296993792057e-08, 1.1052100993692875e-08, 1.1550979986786842e-08, 1.1028691977262497e-08]


In [27]:
from statistics import mean, stdev

print(f"mean {mean(result) * 1000000000} ns")
print(f"stdev {stdev(result) * 1000000000} ns")

mean 11.30102599305766 ns
stdev 0.31787355736488865 ns


In [28]:
a = 1
timeit.repeat("stmt=a+1", number=number, repeat=repeat)

NameError: name 'a' is not defined

In [29]:
a = 1
print(timeit.repeat("stmt=a+1", number=number, repeat=repeat, globals=globals()))

[2.256257101893425, 2.2410292997956276, 2.1918498016893864, 2.1492957025766373, 2.1939339004456997, 2.156707100570202, 2.1734426990151405]


In [30]:
def b():
    c = 1
    print(timeit.repeat("stmt=c+1", number=number, repeat=repeat, globals=globals()))    
    
b()

NameError: name 'c' is not defined

In [31]:
def b():
    c = 1
    print(timeit.repeat("stmt=c+1", number=number, repeat=repeat, globals=locals()))    
    
b()

[2.2663614004850388, 2.2158589027822018, 2.2128161005675793, 2.1701188012957573, 2.165589399635792, 2.216084398329258, 2.170499600470066]


In [38]:
import timeit
from statistics import mean, stdev

def benchmark(mbparams={}, match_num=50000, seed=0, number=10, repeat=7):
    if seed is not None:
        random.seed(seed)       
        
    from ai import ai2, ai14s, ai_match, ai_abs_dls

    ai_match(ai=[ai2, ai2], match_num=match_num, mbparams=mbparams)   
    ai_match(ai=[ai14s, ai2], match_num=match_num, mbparams=mbparams)

    mb = Marubatsu(**mbparams)
    eval_params = {"minimax": True}
    stmt = "ai_abs_dls(mb, eval_func=ai14s, eval_params=eval_params, use_tt=True, maxdepth=8)"
    print("ai_abs_dls")
    result = timeit.repeat(stmt=stmt, number=number, repeat=repeat, globals=locals())
    result = [time / number for time in result]
    print(f"{mean(result) * 1000:5.1f} ms ± {stdev(result) * 1000:5.1f} ms per loop (mean ± std. dev. of {repeat} runs, {number} loops each)")

In [39]:
benchmark()

ai2 VS ai2


100%|██████████| 50000/50000 [00:04<00:00, 11415.43it/s]


count     win    lose    draw
o       29454   14352    6194
x       14208   29592    6200
total   43662   43944   12394

ratio     win    lose    draw
o       58.9%   28.7%   12.4%
x       28.4%   59.2%   12.4%
total   43.7%   43.9%   12.4%

ai14s VS ai2


100%|██████████| 50000/50000 [00:52<00:00, 955.25it/s] 


count     win    lose    draw
o       49446       0     554
x       44043       0    5957
total   93489       0    6511

ratio     win    lose    draw
o       98.9%    0.0%    1.1%
x       88.1%    0.0%   11.9%
total   93.5%    0.0%    6.5%

ai_abs_dls
 19.1 ms ±   2.3 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
